In [27]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

from torchvision.models import ResNet18_Weights, resnet18
from tqdm.auto import tqdm
from torchvision import datasets, transforms

from src.imagenet_x.evaluate import ImageNetX, get_vanilla_transform, ImageNetXImageFolder, load_annotations
from src.imagenet_x import FACTORS, plots
import os

import shutil

In [21]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
imagenet_val_path = '../data/ImageNetVal'
transforms = get_vanilla_transform()
dataset = ImageNetX(imagenet_val_path, which_factor='top', partition='val', transform=transforms, filter_prototypes=False)    

In [34]:
class TopFactor(datasets.ImageFolder):
    
    def __init__(self, folder_path, *args, **kwargs):
        super().__init__(folder_path, *args, **kwargs)

datafolder = TopFactor('/media/juliu/T7/TopFactor/')

In [40]:

normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]
)

In [78]:
def get_new_annotations(which_factor = 'top', filter_prototypes=False, partition='val'):
    annotations = load_annotations(which_factor=which_factor, partition=partition, filter_prototypes=filter_prototypes)

    if partition == 'val':
        #rows with errors:
        #17758
        #29800
        annotations = annotations.drop([17758, 29800]).reset_index(drop=True)

    if partition == 'train':
        #rows with errors:
        errors = [1760, 2197, 5087, 6654, 10510, 10960]
        annotations = annotations.drop(errors).reset_index(drop=True)
    
    filenames = pd.DataFrame(annotations['file_name'])
    annotations = annotations.drop(columns=['class', 'justification', 'one_word', 'metaclass'])
    classes = pd.from_dummies(annotations.drop(columns=['file_name']))
    
    annotations = filenames.join(classes)
    annotations.columns = ['file_name', 'class']

    return annotations

In [54]:
dftrain = get_new_annotations(partition='val')

In [79]:
dfval = get_new_annotations(partition='train')

In [84]:
dfval['nclass'] = dfval['file_name'].apply(lambda x: x.split('_')[0])

In [14]:
idx_to_class = {
    0: 'multiple_objects',
    1: 'background',
    2: 'color',
    3: 'brighter',
    4: 'darker',
    5: 'style',
    6: 'larger',
    7: 'smaller',
    8: 'object_blocking',
    9: 'person_blocking',
    10: 'partial_view',
    11: 'pattern',
    12: 'pose',
    13: 'shape',
    14: 'subcategory',
    15: 'texture'
}

class_to_idx = {v: k for k, v in idx_to_class.items()}

In [81]:
[os.makedirs(f'/media/juliu/T7/TopFactor/val/{key}') for key in class_to_idx.keys()]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [33]:

for i,item in df5.iterrows():
    itemclass = item[1]
    itemname = item[0]
    shutil.copy(f'{path}ImageNetVal/val/{itemname}', f'{path}TopFactor/{itemclass}/{itemname}')

In [86]:
dfval

,file_name,class,nclass
0,n03633091_15528.JPEG,background,n03633091
1,n02950826_6852.JPEG,background,n02950826
2,n03404251_11352.JPEG,background,n03404251
3,n03930313_3341.JPEG,pattern,n03930313
4,n02099267_827.JPEG,pose,n02099267
...,...,...,...
11222,n03599486_3557.JPEG,pose,n03599486
11223,n03710637_762.JPEG,pattern,n03710637
11224,n02088466_3728.JPEG,pose,n02088466
11225,n02113023_11057.JPEG,background,n02113023


In [88]:
path = '/media/juliu/T7/'

for i,item in dfval.iterrows():
    itemname = item[0]
    itemclass = item[1]
    itemnclass = item[2]
    shutil.copy(f'{path}ImageNetRaw/train/{itemnclass}/{itemname}', f'{path}TopFactor/val/{itemclass}/{itemname}')

In [89]:
val = TopFactor('/media/juliu/T7/TopFactor/val', transform=transform)

In [90]:
val[0]

(tensor([[[-1.5528, -1.7925, -1.7069,  ..., -0.4397, -0.1314, -0.5424],
          [-1.2617, -1.4672, -1.6042,  ..., -0.2856,  0.0569, -0.5938],
          [-1.0904, -1.0219, -1.1589,  ..., -0.2171,  0.0227, -0.6109],
          ...,
          [-1.5014, -1.4843, -1.4843,  ..., -1.7583, -1.5357, -1.4672],
          [-1.4672, -1.3987, -1.3815,  ..., -1.7754, -1.5870, -1.4843],
          [-1.9124, -1.8268, -1.7754,  ..., -1.8097, -1.5185, -1.5014]],
 
         [[-1.2829, -1.4930, -1.4055,  ..., -0.3025,  0.0476, -0.5476],
          [-0.9678, -1.1779, -1.2304,  ..., -0.1625,  0.1877, -0.5301],
          [-0.7927, -0.7577, -0.8978,  ..., -0.1099,  0.1527, -0.4776],
          ...,
          [-0.7052, -0.7402, -0.7577,  ..., -0.9503, -0.4601, -0.4776],
          [-0.8627, -0.8277, -0.7402,  ..., -0.9328, -0.4951, -0.4951],
          [-1.7031, -1.6506, -1.5455,  ..., -0.8803, -0.5126, -0.5301]],
 
         [[-1.2641, -1.6302, -1.4733,  ..., -0.4798, -0.2010, -0.5844],
          [-1.1421, -1.2816,